Defining all methods, classes, and objects

In [4]:
import json 
import os

class Phonenumber():
    def __init__(self, cellphone = None, home = None, work = None):
        self.cellphone = cellphone
        self.home = home
        self.work = work
    def to_dict(self):
        return {
            "cellphone": self.cellphone, 
            "home": self.home,
            "work": self.work
        }
    @classmethod
    def from_dict(cls, data):
        return cls(
            cellphone = data.get("cellphone"),
            home = data.get("home"),
            work = data.get("work")
        )
def get_phone_input(prompt):
    value = input(prompt)
    if value.strip() == "":
        return None
    else: 
        return value
class Contact():
    def __init__(self, firstname, lastname, phone: Phonenumber, city):
        self.firstname = firstname
        self.lastname = lastname
        self.phone = phone
        self.city = city
        
    def to_dict(self):
        return {
            "firstname": self.firstname,
            "lastname": self.lastname,
            "phone": self.phone.to_dict(),
            "city": self.city
        }
    @classmethod
    def from_dict(cls, data):
        phone = Phonenumber.from_dict(data.get("phone", {}))
        return cls(
            firstname=data.get("firstname"),
            lastname=data.get("lastname"),
            phone=phone,
            city=data.get("city")
        )
            
class Phonebook():
    def __init__(self):
        self.contacts = []
        self.search_counts = {}
    def add_contact(self, contact = Contact):
        self.contacts.append(contact)
    def remove_contact(self, firstname, lastname):
        self.contacts = [j for j in self.contacts if not (j.firstname == firstname and j.lastname == lastname)]
                         
    def edit_contact(self):
        firstname = input("Enter the first name of the contact you want to edit: ")
        lastname = input("Enter the last name of the contact you want to edit: ")
        found = False
        for b in self.contacts:
            if b.firstname.lower() == firstname.lower() and b.lastname.lower() == lastname.lower():
                print("Enter their new information: ")
                new_first = input("Enter their new first name: ")
                new_last = input("Enter their new last name: ")
                cellphone = get_phone_input("Enter new cellphone number (press enter to skip): ")
                home = get_phone_input("Enter new home phone number (press enter to skip): ")
                work = get_phone_input("Enter new work phone number (press enter to skip): ")
                new_city = input("Enter  the new city of residence: ")
                if new_first:
                    b.firstname = new_first
                if new_last: 
                    b.lastname = new_last
                if cellphone:
                    b.phone.cellphone = cellphone
                if home:
                    b.phone.home = home
                if work:
                    b.phone.work = work
                if new_city:
                    b.city = new_city
                found = True
                print("Info updated!")
                break
        if not found:
            print("No such contact was found!")

    def search_contact(self, keyword):
        results = []
        for j in self.contacts:
            if (keyword.lower() in j.lastname.lower() or
                keyword.lower() in j.firstname.lower() or
                keyword.lower() in j.city.lower()or
                (j.phone.cellphone and keyword in j.phone.cellphone) or
                (j.phone.work and keyword in j.phone.work) or
                (j.phone.home and keyword in j.phone.home)):
                results.append(j)
                fullname = f"{j.firstname} {j.lastname}".lower()
                self.search_counts[fullname] = self.search_counts.get(fullname,0) + 1
        return results
    def show_in_order(self):
        ordered_contacts = sorted(self.contacts, key= lambda c: c.lastname.lower())
        print("Contacts in alphabetical order: ")
        for contact in ordered_contacts:
            print(f"{contact.firstname} {contact.lastname} - {contact.city}")
            print(f" Phone numbers: ")
            there_is_a_number = False
            if contact.phone.cellphone:
                print(f"cellphone number: {contact.phone.cellphone}")
                there_is_a_number = True
            if contact.phone.home:
                print(f"home phone number: {contact.phone.home}")
                there_is_a_number = True
            if contact.phone.work:
                print(f"work phone number: {contact.phone.work}")
                there_is_a_number = True
            if not there_is_a_number:
                print("No number available for this contact!")
    def show_phone_types(self, phone_type):
        valid_input = ["home", "work", "cellphone"]
        if phone_type.lower() not in valid_input:
            print("Input doesn't match any categories!")
            return
        print(f"Contacts with {phone_type} number: ")
        for c in self.contacts: 
            number = getattr(c.phone, phone_type)
            if number:
                print(f"{c.firstname} {c.lastname} {phone_type}: {number}")
    def save_to_file(self, filename):
        data = [c.to_dict() for c in self.contacts]
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)
    def load_from_file(self, filename):
        try:
            with open(filename) as f:
                data = json.load(f)
                self.contacts = [Contact.from_dict(item) for item in data]
        except FileNotFoundError:
            print(f"No file named {filename} found")
            self.contacts = []
                
filename = os.path.join("C:\\Users","h.rahnavard","Downloads", "contacts.json.txt")
phonebook = Phonebook()
phonebook.load_from_file(filename)                 

Menu and user functions:

In [5]:
while True:
    print("What do you want to do?")
    print("1-Add a contact")
    print("2-remove a contact")
    print("3-change a contact's info")
    print("4-Search a contact")
    print("5-look up how many times a contact has been searched")
    print("6-sort the contacts")
    print("7-None, exit")
    user_choice = input("Choose an option")
    if user_choice == "1":
        firstname = input("Enter their first name: ")
        lastname = input("Enter their lats name: ")
        city = input("Enter their city of residence: ")
        cellphone = get_phone_input("Enter their cellphone number (or press enter to skip): ")
        home = get_phone_input("Enter their home phone number (or press enter to skip): ")                          
        work = get_phone_input("Enter their work phone number (or press enter to skip): ")
        phone = Phonenumber(cellphone, home, work)
        contact = Contact(firstname, lastname, phone, city)
        phonebook.add_contact(contact)
    elif user_choice == "2":
        firstname = input("Enter their first name: ")
        lastname = input("Enter their last name: ")
        phonebook.remove_contact(firstname, lastname)
    elif user_choice == "3":
        phonebook.edit_contact()
    elif user_choice == "4":
        keyword = input("Who or what number are you looking for?")
        results = phonebook.search_contact(keyword)
        if results:
            for i in results:
                print(f"{i.firstname} {i.lastname} - {i.city}")
                print(f"{i.firstname} {i.lastname}\'s phone numbers: ")
                if i.phone.cellphone:
                    print(f"cellphone number: {i.phone.cellphone}")
                if i.phone.home:
                    print(f"home phone number: {i.phone.home}")
                if i.phone.work:
                    print(f"work phone number: {i.phone.work}")
        else:
            print("No such contact found!")
    elif user_choice == "5":
        firstname = input("Enter their first name: ")
        lastname = input("Enter their last name: ")
        fullname = f"{firstname} {lastname}".lower()
        if fullname in phonebook.search_counts:
            print(f"The number of times they have been searched: {phonebook.search_counts[fullname]}")
        else: 
            print("They have not been searched before.")
    elif user_choice == "6":
        print("What method of sorting do you want? ")
        print("1-Sort by name")
        print("2-sort by last name")
        print("3-sort by telephone number")
        print("4-sort by city of residence")
        print("5-sort by phone categories")
        user_choice2 = input("Choose your method")
        if user_choice2 == "1":
            ordered_contacts2 = sorted(phonebook.contacts, key= lambda c: c.firstname.lower())
            print("Contacts sorted by their first names: ")
            for contact in ordered_contacts2:
                print(f"{contact.firstname} {contact.lastname} - {contact.city}")
                print(f" Phone numbers: ")
                there_is_a_number = False
                if contact.phone.cellphone:
                    print(f"cellphone number: {contact.phone.cellphone}")
                    there_is_a_number = True
                if contact.phone.home:
                    print(f"home phone number: {contact.phone.home}")
                    there_is_a_number = True
                if contact.phone.work:
                    print(f"work phone number: {contact.phone.work}")
                    there_is_a_number = True
                if not there_is_a_number:
                    print("No number available for this contact!")
        elif user_choice2 == "2":
            ordered_contacts3 = sorted(phonebook.contacts, key= lambda c: c.lastname.lower())
            print("Contacts sorted by their last names: ")
            for contact in ordered_contacts3:
                print(f"{contact.firstname} {contact.lastname} - {contact.city}")
                print(f" Phone numbers: ")
                there_is_a_number = False
                if contact.phone.cellphone:
                    print(f"cellphone number: {contact.phone.cellphone}")
                    there_is_a_number = True
                if contact.phone.home:
                    print(f"home phone number: {contact.phone.home}")
                    there_is_a_number = True
                if contact.phone.work:
                    print(f"work phone number: {contact.phone.work}")
                    there_is_a_number = True
                if not there_is_a_number:
                    print("No number available for this contact!")
        elif user_choice2 == "3":
            ordered_contacts4 = sorted(phonebook.contacts, key= lambda c: (c.phone.cellphone or "", c.phone.home or "", c.phone.work or ""))
            print("Contacts sorted by their phone number: ")
            for contact in ordered_contacts4:
                print(f"{contact.firstname} {contact.lastname} - {contact.city}")
                print(f" Phone numbers: ")
                there_is_a_number = False
                if contact.phone.cellphone:
                    print(f"cellphone number: {contact.phone.cellphone}")
                    there_is_a_number = True
                if contact.phone.home:
                    print(f"home phone number: {contact.phone.home}")
                    there_is_a_number = True
                if contact.phone.work:
                    print(f"work phone number: {contact.phone.work}")
                    there_is_a_number = True
                if not there_is_a_number:
                    print("No number available for this contact!")
        elif user_choice2 == "4":
            ordered_contacts5 = sorted(phonebook.contacts, key= lambda c: c.city.lower())
            print("Contacts sorted by their city of residence: ")
            for contact in ordered_contacts5:
                print(f"{contact.firstname} {contact.lastname} - {contact.city}")
                print(f" Phone numbers: ")
                there_is_a_number = False
                if contact.phone.cellphone:
                    print(f"cellphone number: {contact.phone.cellphone}")
                    there_is_a_number = True
                if contact.phone.home:
                    print(f"home phone number: {contact.phone.home}")
                    there_is_a_number = True
                if contact.phone.work:
                    print(f"work phone number: {contact.phone.work}")
                    there_is_a_number = True
                if not there_is_a_number:
                    print("No number available for this contact!")
        elif user_choice2 == "5":
            category = input("What phone type are you looking for?")
            phonebook.show_phone_types(category)
        else: 
            print("Invalid input")
    elif user_choice == "7":
        print("Saving Data...")
        phonebook.save_to_file(filename)
        break
    
    

What do you want to do?
1-Add a contact
2-remove a contact
3-change a contact's info
4-Search a contact
5-look up how many times a contact has been searched
6-sort the contacts
7-None, exit


Choose an option 1
Enter their first name:  Ali
Enter their lats name:  Hassani
Enter their city of residence:  Mashhad
Enter their cellphone number (or press enter to skip):  928398
Enter their home phone number (or press enter to skip):  
Enter their work phone number (or press enter to skip):  2873298713


What do you want to do?
1-Add a contact
2-remove a contact
3-change a contact's info
4-Search a contact
5-look up how many times a contact has been searched
6-sort the contacts
7-None, exit


Choose an option 7


Saving Data...
